# LIBRARY

In [1]:
from keras.models import Sequential    #librerie per CNN
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras.layers.core import Dense, Activation
from keras import layers
from keras.optimizers import SGD

from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras import optimizers



from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,AveragePooling2D
from keras.layers import Flatten
from keras.layers import Dense

from keras.callbacks import ModelCheckpoint #choose best model


import numpy as np
import pandas as pd

Using TensorFlow backend.


# PROVA GPU

In [2]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
with tf.Session() as sess:
    print (sess.run(c))
    
    
#se non fa errore vuol dire che sta andando con la GPU

[[22. 28.]
 [49. 64.]]


# F1 - SCORE

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# DATI

In [13]:
train_dim=80000
test_dim=20000
n=train_dim+test_dim
epoche=200
larg=60
alt=30
batch=32

dataset='immagini'
classi=['down','up']
tipo_col="grayscale"
lear_rate=1e-5

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10378688447122119378
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4943878553
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4037778772904131608
physical_device_desc: "device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
with tf.device('/gpu:0'):
    classifier = Sequential()

    classifier.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',
                     input_shape = (larg, alt, 1),strides=1, padding='valid'))
    classifier.add(BatchNormalization())
    classifier.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',strides=1, padding='valid'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(layers.Dropout(0.60))

    classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',strides=1, padding='valid'))
    classifier.add(BatchNormalization())
    classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',strides=1, padding='valid'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPooling2D(pool_size = (2, 2)))
    classifier.add(layers.Dropout(0.60))





    classifier.add(Flatten())
    classifier.add(Dense(128, activation='relu'))
    classifier.add(layers.Dropout(0.25))
    classifier.add(Dense(256, activation='relu'))
    classifier.add(layers.Dropout(0.60))
    classifier.add(Dense(len(classi), activation='softmax'))
    classifier.summary()
    
    from keras.optimizers import Adam

    optimizer = Adam(lr=lear_rate)
    classifier.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


    # Part 2 - Fitting the CNN to the images

    from keras.preprocessing.image import ImageDataGenerator

    #applichiamo delle traformazioni casuacategorical_crossentropyli per aumentare il numero di esempi nel training
    train_datagen = ImageDataGenerator(rescale = 1./255)

    test_datagen = ImageDataGenerator(rescale = 1./255)

    training_set = train_datagen.flow_from_directory(r'C:\Users\user\Desktop\Data_Science\CNN\immagini\training',
                                                     target_size = (larg, alt), #dimensione in ingresso alla cnn
                                                     color_mode=tipo_col,  #se rgb o grayscale
                                                     classes = classi, #classi in cui farà predizione
                                                     batch_size = batch,
                                                     class_mode = 'categorical',#se la classe è binaria o meno 
                                                     shuffle=True)  #se pesca casuamente il dato dal dataset

    # NB per aumentare ulteriormente le perfomance si potrebbe aumentare la dimensione delle immagini in ingresso,
    # tuttavia il tempo necessario all'esecuzione aumenta notevolemente                                                 

    #ripetiamo la stessa cosa anche per il test set        
    test_set = test_datagen.flow_from_directory(r'C:\Users\user\Desktop\Data_Science\CNN\immagini\testing',
                                                target_size = (larg, alt),
                                                batch_size = batch,
                                                color_mode=tipo_col,
                                                classes = classi,
                                               class_mode = 'categorical',
                                               shuffle=True)
                    #CHECKPOINT salva sul disco, i modelli migliori
    #early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, mode='min', verbose=1)
    checkpoint = ModelCheckpoint('model_best_weights.h5', monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)


    classifier.fit_generator(training_set,
                             steps_per_epoch = train_dim,   #numero di immagini nel trainig set (tutte le osservazioni vengono passate durante ogni epoca)
                             epochs = epoche,
                             validation_data = test_set,
                             validation_steps = test_dim,verbose=1,callbacks = [checkpoint])   #corrisponde al numero di immagini nel mio test set
with tf.Session() as sess:
    print (sess.run(c))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 58, 28, 16)        160       
_________________________________________________________________
batch_normalization_29 (Batc (None, 58, 28, 16)        64        
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 56, 26, 16)        2320      
_________________________________________________________________
batch_normalization_30 (Batc (None, 56, 26, 16)        64        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 28, 13, 16)        0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 28, 13, 16)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 26, 11, 32)        4640      
__________

80000/80000 [==============================] - 1252s 16ms/step - loss: 0.6894 - acc: 0.5346 - val_loss: 0.6933 - val_acc: 0.5127

Epoch 00024: loss improved from 0.68959 to 0.68939, saving model to model_best_weights.h5
Epoch 25/200
80000/80000 [==============================] - 1254s 16ms/step - loss: 0.6891 - acc: 0.5360 - val_loss: 0.6935 - val_acc: 0.5118

Epoch 00025: loss improved from 0.68939 to 0.68909, saving model to model_best_weights.h5
Epoch 26/200
80000/80000 [==============================] - 1253s 16ms/step - loss: 0.6888 - acc: 0.5369 - val_loss: 0.6939 - val_acc: 0.5131

Epoch 00026: loss improved from 0.68909 to 0.68877, saving model to model_best_weights.h5
Epoch 27/200
80000/80000 [==============================] - 1257s 16ms/step - loss: 0.6885 - acc: 0.5382 - val_loss: 0.6940 - val_acc: 0.5133

Epoch 00027: loss improved from 0.68877 to 0.68851, saving model to model_best_weights.h5
Epoch 28/200
80000/80000 [==============================] - 1253s 16ms/step - los

80000/80000 [==============================] - 1252s 16ms/step - loss: 0.6586 - acc: 0.5958 - val_loss: 0.7061 - val_acc: 0.5060

Epoch 00059: loss improved from 0.66021 to 0.65862, saving model to model_best_weights.h5
Epoch 60/200
80000/80000 [==============================] - 1251s 16ms/step - loss: 0.6576 - acc: 0.5979 - val_loss: 0.7064 - val_acc: 0.5075

Epoch 00060: loss improved from 0.65862 to 0.65755, saving model to model_best_weights.h5
Epoch 61/200
80000/80000 [==============================] - 1256s 16ms/step - loss: 0.6564 - acc: 0.5997 - val_loss: 0.7070 - val_acc: 0.5065

Epoch 00061: loss improved from 0.65755 to 0.65638, saving model to model_best_weights.h5
Epoch 62/200
80000/80000 [==============================] - 1251s 16ms/step - loss: 0.6547 - acc: 0.6019 - val_loss: 0.7075 - val_acc: 0.5062

Epoch 00062: loss improved from 0.65638 to 0.65473, saving model to model_best_weights.h5
Epoch 63/200
80000/80000 [==============================] - 1252s 16ms/step - los

# PROVA RETE

In [ ]:
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> [Dense -> Dropout]*2 -> Softmax
"""
classifier = Sequential()

classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu',
                 input_shape = (larg, alt, 1)))
classifier.add(BatchNormalization())
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(layers.Dropout(0.40))

classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(layers.Dropout(0.40))


classifier.add(Flatten())
classifier.add(Dense(512, activation='relu'))
classifier.add(layers.Dropout(0.25))
classifier.add(Dense(1024, activation='relu'))
classifier.add(layers.Dropout(0.50))
classifier.add(Dense(2, activation='softmax'))
classifier.summary()

"""

# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> [Dense -> Dropout]*2 -> Softmax

classifier = Sequential()

classifier.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',
                 input_shape = (larg, alt, 1),strides=1, padding='valid'))
classifier.add(BatchNormalization())
classifier.add(Conv2D(filters = 16, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',strides=1, padding='valid'))
classifier.add(BatchNormalization())
classifier.add(AveragePooling2D(pool_size = (2, 2)))
classifier.add(layers.Dropout(0.60))

classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',strides=1, padding='valid'))
classifier.add(BatchNormalization())
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), activation='relu',kernel_initializer='he_normal',strides=1, padding='valid'))
classifier.add(BatchNormalization())
classifier.add(AveragePooling2D(pool_size = (2, 2)))
classifier.add(layers.Dropout(0.60))





classifier.add(Flatten())
classifier.add(Dense(128, activation='relu'))
classifier.add(layers.Dropout(0.25))
classifier.add(Dense(256, activation='relu'))
classifier.add(layers.Dropout(0.60))
classifier.add(Dense(2, activation='softmax'))
classifier.summary()







In [ ]:
from keras.optimizers import Adam

optimizer = Adam(lr=lear_rate)
classifier.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

#applichiamo delle traformazioni casuacategorical_crossentropyli per aumentare il numero di esempi nel training
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(dataset+'/training',
                                                 target_size = (larg, alt), #dimensione in ingresso alla cnn
                                                 color_mode=tipo_col,  #se rgb o grayscale
                                                 classes = classi, #classi in cui farà predizione
                                                 batch_size = batch,
                                                 class_mode = 'categorical',#se la classe è binaria o meno 
                                                 shuffle=True)  #se pesca casuamente il dato dal dataset
                                                 
# NB per aumentare ulteriormente le perfomance si potrebbe aumentare la dimensione delle immagini in ingresso,
# tuttavia il tempo necessario all'esecuzione aumenta notevolemente                                                 

#ripetiamo la stessa cosa anche per il test set        
test_set = test_datagen.flow_from_directory(dataset+'/training',
                                            target_size = (larg, alt),
                                            batch_size = batch,
                                            color_mode=tipo_col,
                                            classes = classi,
                                           class_mode = 'categorical',
                                           shuffle=True)
                #CHECKPOINT salva sul disco, i modelli migliori
#early_stop = EarlyStopping(monitor='loss', min_delta=0.001, patience=3, mode='min', verbose=1)
checkpoint = ModelCheckpoint('model_best_weights.h5', monitor='loss', verbose=1, save_best_only=True, mode='min', period=1)
                             
                             
classifier.fit_generator(training_set,
                         steps_per_epoch = train_dim,   #numero di immagini nel trainig set (tutte le osservazioni vengono passate durante ogni epoca)
                         epochs = epoche,
                         validation_data = test_set,
                         validation_steps = test_dim,verbose=1,callbacks = [checkpoint])   #corrisponde al numero di immagini nel mio test set

In [ ]:
#Initialising the CNN
classifier = Sequential()



# Step 1 - Convolution
classifier.add(Conv2D(96, (3, 3), input_shape = (larg, alt, 1), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(96, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(96, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))


# Adding a second convolutional layer
classifier.add(Conv2D(192, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(384, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(layers.Dropout(0.5))


classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 96*3, activation = 'relu'))
classifier.add(layers.Dropout(0.3))

classifier.add(Dense(units = 3, activation = 'softmax'))

from keras.optimizers import SGD
#opt = SGD(lr=0.1)
opt=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

classifier.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy',f1])



# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

#applichiamo delle traformazioni casuacategorical_crossentropyli per aumentare il numero di esempi nel training
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training',
                                                 target_size = (larg, alt), #dimensione in ingresso alla cnn
                                                 color_mode="grayscale",
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 shuffle=False) #se la classe è binaria o meno 
                                                 
# NB per aumentare ulteriormente le perfomance si potrebbe aumentare la dimensione delle immagini in ingresso,
# tuttavia il tempo necessario all'esecuzione aumenta notevolemente                                                 

#ripetiamo la stessa cosa anche per il test set        
test_set = test_datagen.flow_from_directory('dataset/testing',
                                            target_size = (larg, alt),
                                            batch_size = 32,
                                            color_mode="grayscale",
                                           class_mode = 'categorical',
                                           shuffle=False)

history=classifier.fit_generator(training_set,
                         steps_per_epoch = train_dim-200000,   #numero di immagini nel trainig set (tutte le osservazioni vengono passate durante ogni epoca)
                         epochs = epoche-40,
                         validation_data = test_set,
                         validation_steps = test_dim-100000,verbose=1,workers=4)   #corrisponde al numero di immagini nel mio test set

In [ ]:
#Initialising the CNN
classifier = Sequential()



# Step 1 - Convolution
classifier.add(Conv2D(60, (3, 3), input_shape = (larg, alt, 1), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(40, (3, 3), activation = 'relu',kernel_initializer='he_normal'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(20, (3, 3), activation = 'relu',kernel_initializer='he_normal'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(16, (3, 3), activation = 'relu',kernel_initializer='he_normal'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
#classifier.add(layers.Dropout(0.4))


classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 20, activation = 'relu',kernel_initializer='he_normal'))
classifier.add(layers.Dropout(0.5))

classifier.add(Dense(units = 3, activation = 'softmax'))

from keras.optimizers import SGD
opt = SGD(lr=0.000000005)
classifier.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy',f1])



# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

#applichiamo delle traformazioni casuali per aumentare il numero di esempi nel training
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training',
                                                 target_size = (larg, alt), #dimensione in ingresso alla cnn
                                                 color_mode="grayscale",
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 shuffle=False) #se la classe è binaria o meno 
                                                 
# NB per aumentare ulteriormente le perfomance si potrebbe aumentare la dimensione delle immagini in ingresso,
# tuttavia il tempo necessario all'esecuzione aumenta notevolemente                                                 

#ripetiamo la stessa cosa anche per il test set        
test_set = test_datagen.flow_from_directory('dataset/testing',
                                            target_size = (larg, alt),
                                            batch_size = 32,
                                            color_mode="grayscale",
                                           class_mode = 'categorical',
                                           shuffle=False)

history=classifier.fit_generator(training_set,
                         steps_per_epoch = train_dim,   #numero di immagini nel trainig set (tutte le osservazioni vengono passate durante ogni epoca)
                         epochs =epoche+40,
                         validation_data = test_set,
                         validation_steps = test_dim,verbose=1,workers=4)   #corrisponde al numero di immagini nel mio test set

In [ ]:
# from keras.layers import LeakyReLU


seed = 7

#Initialising the CNN
model = Sequential()


model.add(Conv2D(30, 20, 20, activation = 'relu', input_shape = (larg, alt, 1)))
model.add(BatchNormalization())

model.add(Conv2D(10, 20, 20, activation = 'relu'))
model.add(BatchNormalization())

model.add(Conv2D(10, 10, 10, activation = 'relu'))
model.add(BatchNormalization())

model.add(Conv2D(5, 10, 10, activation = 'relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(3, 10, 10, activation = 'relu'))

model.add(MaxPooling2D(pool_size=(4, 4)))

# Flatten the 3D output to 1D tensor for a fully connected layer to accept the input
model.add(Flatten())
model.add(Activation("relu"))


model.add(Dense(2, activation = 'softmax')) #Last layer with one output per class

# The function to optimize is the cross entropy between the true label and the output (softmax) of the model
# We will use adadelta to do the gradient descent see http://cs231n.github.io/neural-networks-3/#ada
from keras.optimizers import SGD




# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

#applichiamo delle traformazioni casuali per aumentare il numero di esempi nel training
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training',
                                                 target_size = (larg, alt), #dimensione in ingresso alla cnn
                                                 color_mode="grayscale",
                                                 batch_size = 512,
                                                 class_mode = 'categorical',
                                                 shuffle=True) #se la classe è binaria o meno 
                                                 
# NB per aumentare ulteriormente le perfomance si potrebbe aumentare la dimensione delle immagini in ingresso,
# tuttavia il tempo necessario all'esecuzione aumenta notevolemente                                                 

#ripetiamo la stessa cosa anche per il test set        
test_set = test_datagen.flow_from_directory('dataset/testing',
                                            target_size = (larg, alt),
                                            batch_size = 512,
                                            color_mode="grayscale",
                                           class_mode = 'categorical',
                                           shuffle=True)
#Time-Based Learning Rate Schedule


#epochs = 10
#learning_rate = 0.01
#decay_rate = learning_rate / epochs
#momentum = 0.8
#sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# Fit the model

#Drop-Based Learning Rate Schedule



opt=optimizers.Adagrad(lr=0.00005, epsilon=1e-08, decay=0.0)
#keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
#keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
#opt=optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)





print(model.summary())

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy',f1])
# learning schedule callback

filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint("result/"+filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

history=model.fit_generator(training_set,
                         steps_per_epoch = train_dim,   #numero di immagini nel trainig set (tutte le osservazioni vengono passate durante ogni epoca)
                         epochs = epoche+20,
                         validation_data = test_set,
                         validation_steps = test_dim,
                         verbose=1)   #corrisponde al numero di immagini nel mio test set

In [ ]:
#AlwzNet
from keras.layers import ZeroPadding2D
k=1
model = Sequential()
model.add(Conv2D(int(16*k), 5, 5, activation = 'relu', input_shape = (larg, alt, 1)))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(16*k), 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(32*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(32*k), 3, 3, activation='relu'))
model.add(layers.Dropout(0.6))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(64*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(64*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(64*k), 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(128*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(128*k), 3, 3, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(128*k), 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(128*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(128*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(128*k), 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(256*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(int(256*k), 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(layers.Dropout(0.6))
model.add(Conv2D(int(256*k), 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))





model.add(Flatten())
model.add(Dense(int(1024*k), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Dense(int(1024*k), activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Dense(2, activation='softmax'))




# The function to optimize is the cross entropy between the true label and the output (softmax) of the model
# We will use adadelta to do the gradient descent see http://cs231n.github.io/neural-networks-3/#ada
from keras.optimizers import SGD




# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

#applichiamo delle traformazioni casuali per aumentare il numero di esempi nel training
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training',
                                                 target_size = (larg, alt), #dimensione in ingresso alla cnn
                                                 color_mode="grayscale",
                                                 batch_size = batch,
                                                 class_mode = 'categorical',
                                                 shuffle=False) #se la classe è binaria o meno 
                                                 
# NB per aumentare ulteriormente le perfomance si potrebbe aumentare la dimensione delle immagini in ingresso,
# tuttavia il tempo necessario all'esecuzione aumenta notevolemente                                                 

#ripetiamo la stessa cosa anche per il test set        
test_set = test_datagen.flow_from_directory('dataset/testing',
                                            target_size = (larg, alt),
                                            batch_size = batch,
                                            color_mode="grayscale",
                                           class_mode = 'categorical',
                                           shuffle=False)
#Time-Based Learning Rate Schedule


#epochs = 10
#learning_rate = 0.01
#decay_rate = learning_rate / epoche
#momentum = 0.8
#opt = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# Fit the model

#Drop-Based Learning Rate Schedule



#keras.optimizers.Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
#keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0)
#keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
opt=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)





print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy',f1])
# learning schedule callback


history=model.fit_generator(training_set,
                         steps_per_epoch = train_dim,   #numero di immagini nel trainig set (tutte le osservazioni vengono passate durante ogni epoca)
                         epochs = epoche,
                         validation_data = test_set,
                         validation_steps = test_dim,
                         verbose=1)   #corrisponde al numero di immagini nel mio test set

In [ ]:
from keras.models import load_model
model=load_model("model_best_weights.h5")
print("Loaded model from disk")


In [ ]:
print(K.image_data_format()) # print current format
K.set_image_data_format() # set format

In [ ]:
from keras.preprocessing import image
from keras.models import load_model
model=load_model("model_best_weights_59-150000.h5")
print("Loaded model from disk")

In [35]:





punti_up=0
punti_down=0
punti_neutral=0
ktup=0
ktdown=0
ktneu=0

rag=1000000
l=[]
print("primo")
for raggio in range(0,rag):
    #print(str(raggio/rag))

    try:

        test_image = image.load_img('dataset_new2/training/up/X'+str(raggio)+'.png', target_size = (44, 33),grayscale=True)
        #il passo precedente non basta perchè in ingresso alla rete abbiamo un oggetto di dimensione 64x64x3 (immagine a colori)
        #quindi usiamo il comando successivo per creare il vettore 3D

        test_image = image.img_to_array(test_image)

        test_image = np.expand_dims(test_image, axis = 0)/255


        if model.predict(test_image)[0].argmax() == 0:
            punti_up+=1
        else: pass



        ktup+=1 

    except:

        pass

print("secondo")

for raggio in range(0,rag):
    #print(str(raggio/len(l)))

    try:

        test_image = image.load_img('dataset_new2/training/down/X'+str(raggio)+'.png', target_size = (44, 33),grayscale=True)
        #il passo precedente non basta perchè in ingresso alla rete abbiamo un oggetto di dimensione 64x64x3 (immagine a colori)
        #quindi usiamo il comando successivo per creare il vettore 3D

        test_image = image.img_to_array(test_image)

        test_image = np.expand_dims(test_image, axis = 0)/255


        if model.predict(test_image)[0].argmax() == 1:
            punti_down+=1

        else: pass



        ktdown+=1 

    except:

        pass    

print("terzo")

for raggio in range(0,rag):
    #print(str(raggio/len(l)))

    try:

        test_image = image.load_img('dataset_new2/training/neutral/X'+str(raggio)+'.png', target_size = (44, 33),grayscale=True)
        #il passo precedente non basta perchè in ingresso alla rete abbiamo un oggetto di dimensione 64x64x3 (immagine a colori)
        #quindi usiamo il comando successivo per creare il vettore 3D

        test_image = image.img_to_array(test_image)

        test_image = np.expand_dims(test_image, axis = 0)/255


        if model.predict(test_image)[0].argmax() == 2:
            punti_neutral+=1
        else: pass


        ktneu+=1 

    except:

        pass 




primo
secondo
terzo


In [36]:
acc=(punti_up+punti_dow+punti_neutral)/(ktup+ktdown+ktneu)

In [43]:
punti_neutral/ktneu

0.3044103189938642

In [28]:
(punti_up+punti_dow+punti_neutral)/(((punti_up+punti_dow+punti_neutral)+))

TP/(TP+ FP)

431048

In [8]:
from keras.preprocessing import image
from keras.models import load_model
model=load_model("model_best_weights_044.h5")
print("Loaded model from disk")
test_image = image.load_img('dataset_new2/training/up/X100.png', target_size = (44, 33),grayscale=True)
#il passo precedente non basta perchè in ingresso alla rete abbiamo un oggetto di dimensione 64x64x3 (immagine a colori)
#quindi usiamo il comando successivo per creare il vettore 3D

test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0)/255

Loaded model from disk


In [15]:
model.predict(test_image)[0].argmax()

1